In [43]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score



train_df = pd.read_csv("adult_train.csv")
test_df = pd.read_csv("adult_test.csv")
from sklearn.feature_extraction import DictVectorizer




### Task 1

Step 1

In [3]:
y_train = train_df.target
x_train = train_df.drop("target", axis=1)

y_test = test_df.target
x_test = test_df.drop("target", axis=1)

Step 2

In [22]:
x_train_dicts = x_train.to_dict("records")
x_test_dicts = x_test.to_dict("records")



dv = DictVectorizer()
x_train_encoded = dv.fit_transform(x_train_dicts)
x_test_encoded = dv.transform(x_test_dicts)


In [23]:
clf_forest = RandomForestClassifier()
train_scores = cross_val_score(clf_forest, x_train_encoded, y_train)

Step 3

In [26]:
pipeline = make_pipeline(
  DictVectorizer(),
  RandomForestClassifier()
)

In [46]:
pipeline.fit(x_train_dicts, y_train)
train_preds = pipeline.predict(x_train_dicts)
accuracy_score(y_train, train_preds)

0.9763827892263751

### Task 2

underfitting and overfitting

0.9763827892263751